In [6]:
import pandas as pd
import geopandas as gp
import shapely as sp
from shapely.geometry import Point, Polygon, MultiPolygon
import numpy as np
from itertools import combinations
import intake
from shapely.geometry import MultiPoint

In [7]:
def get_outer_polygon_for_obsid(obsid, return_hull=True):
    tiles = pd.read_csv("tile_coordinates.csv")
    obsid_tiles = tiles.query("obsid == @obsid")
    cols = "BodyFixedCoordinateX BodyFixedCoordinateY".split()
    coords = obsid_tiles[cols]
    coords.columns = ["x", "y"]
    mp = MultiPoint(list(zip(coords.x, coords.y)))
    if return_hull:
        return mp.convex_hull
    else:
        return mp

In [3]:
def interchecker(polygon_array):
    """
    
    """
    bucket2 = []
    combos = []
    iterable1 = range(len(polygon_array)) 
    p = len(polygon_array)
    for n in iterable1:
        if p > 0:
            comb = list(combinations(iterable1,p))
            for item in comb:
                x = item
                combos.append(x)
            p = p-1
        else:
            end

    #the above gives us an array of each possible combination eg
    #combos = [comb[0],comb[1],...]
    #where comb[0] could be (0,1,2,4) or something of the sort
    
    for ele in combos:
        d = {}
        #print(ele) #shows the element used
        #d['region_name'] = region
        #d['index_used'] = ele #for testing
        bucket = []
        for number in ele:
            bucket.append(polygon_array[number])
            #bucket now [poly0, poly1, poly2, poly4] if using comb[0] above
        d['elements_used'] = np.array(bucket)
        if len(bucket)>1:
            y = bucket[0].intersection(bucket[1])
            for i in range(len(bucket)-2):
                y = y.intersection(bucket[i+2])
            #print(y.area)
            d["combo_area"]= y.area
        else:
            #print(bucket[0].area)
            d['combo_area'] = bucket[0].area
        #for number in ele:
            #d['elements_used'] = polygon_array[number]
            #must move elements used to get every element used- where?
        bucket2.append(d)
    return pd.DataFrame(bucket2)

In [4]:
olddata = pd.read_csv("n_obj_df.csv")

In [5]:
olddata.head()

,Latitude,Ls,MY,Which,fraction,n_objects,obsid,pixel_area,region_name
0,-85.4074,214.785,29,both,0.115,933,ESP_012079_0945,2.551911e+07,Macclesfield
1,-85.4065,183.727,29,both,0.070,1971,ESP_011407_0945,2.592302e+06,Macclesfield
2,-85.4056,198.040,29,both,0.072,1525,ESP_011723_0945,8.455380e+06,Macclesfield
3,-85.4049,207.892,29,both,0.168,788,ESP_011934_0945,7.357966e+06,Macclesfield
4,-85.4067,235.282,29,both,0.111,1285,ESP_012501_0945,4.882627e+06,Macclesfield


In [15]:
testregion = 'Macclesfield'

In [52]:
bucket = []
for obs in olddata[olddata['region_name'] == testregion].obsid.unique():
    d = {}
    d['region_name'] = testregion
    d['Obsid_Used'] = obs
    x = get_outer_polygon_for_obsid(obs)
    area = x.area
    d['Obsid_Area'] = area
    d['polygon_shape'] = x
    bucket.append(d)
results = pd.DataFrame(bucket)

In [53]:
results

,Obsid_Area,Obsid_Used,polygon_shape,region_name
0,10.754545,ESP_012079_0945,"POLYGON ((-67.69881540000002 255.7925763, -68....",Macclesfield
1,2.720774,ESP_011407_0945,"POLYGON ((-79.4964947 242.6512937, -71.7773945...",Macclesfield
2,14.759485,ESP_011723_0945,"POLYGON ((-72.54717990000002 244.8776493, -72....",Macclesfield
3,5.338690,ESP_011934_0945,"POLYGON ((-68.00317840000001 256.1826178, -68....",Macclesfield
4,5.246979,ESP_012501_0945,"POLYGON ((-67.989408 256.1117836, -68.32543100...",Macclesfield
5,6.933368,ESP_012290_0945,"POLYGON ((-66.91617959999999 257.8571416, -67....",Macclesfield
6,7.428567,ESP_011406_0945,"POLYGON ((-66.20375920000001 254.0296356, -66....",Macclesfield
7,6.965716,ESP_021494_0945,"POLYGON ((-66.8270493 258.3039062, -66.9954035...",Macclesfield
8,5.204890,ESP_020782_0945,"POLYGON ((-66.48888240000001 259.2801681, -66....",Macclesfield
9,5.208502,ESP_020716_0945,"POLYGON ((-66.41012389999999 259.1488139, -66....",Macclesfield


In [42]:
Testregion_inter = interchecker(results.polygon_shape) #all combos of results

In [72]:
Testregion_inter.head()

,combo_area,elements_used
0,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68..."
1,0.073851,"[POLYGON ((-67.69881540000002 255.7925763, -68..."
2,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68..."
3,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68..."
4,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68..."


In [66]:
len(Testregion_inter.elements_used) #should be half, elements_used nonunique

16383

In [54]:
results.Obsid_Area.max()

14.759485453249887

In [50]:
Testregion_inter.combo_area.max()

14.759485453249887

In [95]:
Testregion_inter.combo_area.head()

0    0.008637
1    0.073851
2    0.008637
3    0.008637
4    0.008637
Name: combo_area, dtype: float64

In [101]:
for polylist in Testregion_inter.elements_used.head():
    print(1)

1
1
1
1
1


In [98]:
for polylist in Testregion_inter.elements_used():
    bucket = []
    for poly in polylist:
        x = df1.obsid[df1['obs_poly'] == poly]
        bucket.append(x)
    bucketlist = list(bucket)
    Testregion_inter['obsids_used'] = bucketlist
#bucket is now each individual polygon
#obsbucket = []
#for ele in bucket:
   # y = df1.obsid[df1['obs_poly'] == ele]
  #  obsbucket.append(y)
#Testregion_inter['obsids_used'] =

TypeError: 'Series' object is not callable

In [16]:
bucket = []
for obs in olddata[olddata['region_name'] == testregion].obsid.unique():
    d = {}
    d['obsid'] = obs #we dont want obs, want obs used in combo
    x = get_outer_polygon_for_obsid(obs)
    d['obs_poly'] = x
    bucket.append(d)
testresult = pd.DataFrame(bucket)

In [22]:
df1 = testresult

In [32]:
df2 = interchecker(testresult.obs_poly)
df2.head()
print(len(df2))
#This won't work, this isn't exactly what I want (re intersectioning)

16383


In [26]:
df1.head()

,obs_poly,obsid
0,"POLYGON ((-67.69881540000002 255.7925763, -68....",ESP_012079_0945
1,"POLYGON ((-79.4964947 242.6512937, -71.7773945...",ESP_011407_0945
2,"POLYGON ((-72.54717990000002 244.8776493, -72....",ESP_011723_0945
3,"POLYGON ((-68.00317840000001 256.1826178, -68....",ESP_011934_0945
4,"POLYGON ((-67.989408 256.1117836, -68.32543100...",ESP_012501_0945


In [57]:
x = 13+78+286+715+1287+1716+1716+1287+715+286+78+13+1

In [60]:
print(x) #we expect this many combinations
print(x*2) #we see this many combinations, getting doubled somehow.

8191
16382
